<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

import keras
from keras.models import load_model

from skimage.io import imshow, imread, imsave
from skimage.color import rgb2grey
from skimage.transform import resize

from tqdm import tqdm

Using TensorFlow backend.


In [2]:
BB_CAPTURE_PATH = '../bb_capture_output/'
TABLE_PATH = '../data/taiku_tables/'

IMG_ROWS, IMG_COLS = 15, 10

In [3]:
folders = next(os.walk(BB_CAPTURE_PATH))[1]
folders.sort()
folders

['bb_capture.capture_2018_03_27_11_53_08',
 'bb_capture.capture_2018_03_27_11_55_29',
 'bb_capture.capture_2018_03_27_11_57_53',
 'bb_capture.capture_2018_03_27_12_27_53',
 'bb_capture.capture_2018_03_27_12_30_18',
 'bb_capture.capture_2018_03_27_12_32_45',
 'bb_capture.capture_2018_03_27_12_35_45',
 'bb_capture.capture_2018_03_27_12_38_33',
 'bb_capture.capture_2018_03_27_12_41_25',
 'bb_capture.capture_2018_03_27_12_44_52',
 'bb_capture.capture_2018_03_27_12_46_54',
 'bb_capture.capture_2018_03_27_12_49_02',
 'bb_capture.capture_2018_03_27_12_51_07',
 'bb_capture.capture_2018_03_27_13_28_02',
 'bb_capture.capture_2018_03_27_13_30_23',
 'bb_capture.capture_2018_03_27_13_32_32',
 'bb_capture.capture_2018_03_27_13_34_43',
 'bb_capture.capture_2018_03_27_13_37_06',
 'bb_capture.capture_2018_03_27_13_39_31',
 'bb_capture.capture_2018_03_27_13_43_06',
 'bb_capture.capture_2018_03_27_13_46_01',
 'bb_capture.capture_2018_03_27_13_48_54',
 'bb_capture.capture_2018_03_27_13_52_15',
 'bb_captur

In [4]:
folders[0]

'bb_capture.capture_2018_03_27_11_53_08'

In [5]:
play_df = pd.read_csv(TABLE_PATH + 'taiko_play.csv')
tmp_df = pd.read_csv(TABLE_PATH + 'taiko_song.csv', dtype={
    'song_length': np.int16
})

play_df = play_df.merge(tmp_df, how='left', left_on='song_id', right_on='song_id')

play_df

,pid,drummer_id,song_id,performance_order,start_time,first_hit_time,song_name,song_length
0,1,1,1,1,3/27/2018 11:53:08,1.522123e+09,夢をかなえてドラえもん,88
1,2,1,1,2,3/27/2018 11:55:29,1.522123e+09,夢をかなえてドラえもん,88
2,3,1,1,3,3/27/2018 11:57:53,1.522123e+09,夢をかなえてドラえもん,88
3,4,1,2,1,3/27/2018 12:27:53,1.522125e+09,ウィーアー！,109
4,5,1,2,2,3/27/2018 12:30:18,1.522125e+09,ウィーアー！,109
5,6,1,2,3,3/27/2018 12:32:45,1.522125e+09,ウィーアー！,109
6,7,1,3,1,3/27/2018 12:35:45,1.522125e+09,六兆年と一夜物語\n,129
7,8,1,3,2,3/27/2018 12:38:33,1.522126e+09,六兆年と一夜物語\n,129
8,9,1,3,3,3/27/2018 12:41:25,1.522126e+09,六兆年と一夜物語\n,129
9,10,1,4,1,3/27/2018 12:44:52,1.522126e+09,残酷な天使のテーゼ,86


In [6]:
model = load_model('model/mnist_model.h5')

X_ANCHOR = 180
Y_ANCHOR = 40
DIGIT_COUNTS = 6

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
def go(row, qq):
    d = datetime.strptime(row['start_time'], '%m/%d/%Y %H:%M:%S')
    directory = d.strftime('bb_capture.capture_%Y_%m_%d_%H_%M_%S')
    score_start_time = float(row['first_hit_time']) - 1
    score_end_time = score_start_time + row['song_length'] + 1

    workspace = '../bb_capture_output/' + directory + '/'

    files = next(os.walk(workspace))[2]
    files.sort()

    score_start_frame = -1
    score_end_frame = -1
    for id_, filename in enumerate(files):
        frametime = float(filename[5:-4])
        if frametime <= score_start_time:
            score_start_frame = id_
        if frametime <= score_end_time:
            score_end_frame = id_
            
    iddd = 0
    for id_, filename in tqdm(enumerate(
            files[score_end_frame - 20:score_end_frame])):
        img = imread(workspace + filename)

        for digit in range(DIGIT_COUNTS):
            cropped = img[X_ANCHOR:X_ANCHOR + IMG_ROWS, Y_ANCHOR +
                          IMG_COLS * digit:Y_ANCHOR + IMG_COLS * (digit + 1)]
            imshow(cropped)
            imsave('images/training_%03d_%04d.png' % (qq, iddd), cropped)
            plt.close()
            iddd += 1

In [8]:
for i in range(len(play_df)):
    row = play_df.iloc[i]
    go(row, i)

0it [00:00, ?it/s]/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_plugins/matplotlib_plugin.py:51: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  out_of_range_float = (np.issubdtype(image.dtype, np.float) and
/usr/bin/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_base.py:1400: MatplotlibDeprecationWarning: The 'box-forced' keyword argument is deprecated since 2.2.
  " since 2.2.", cbook.mplDeprecation)
7it [00:01,  5.25it/s]/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_plugins/matplotlib_plugin.py:74: UserWarning: Low image data range; displaying image with stretched contrast.
  warn("Low image data range; displaying image with "
/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_000_0044.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/bin/anaco

20it [00:03,  5.89it/s]
20it [00:03,  5.67it/s]
20it [00:05,  3.84it/s]
20it [00:03,  5.68it/s]
20it [00:03,  5.74it/s]
9it [00:01,  5.97it/s]/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_026_0054.png is a low contrast image
  warn('%s is a low contrast image' % fname)
20it [00:03,  5.96it/s]
9it [00:01,  5.96it/s]/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_027_0054.png is a low contrast image
  warn('%s is a low contrast image' % fname)
20it [00:03,  5.94it/s]
9it [00:01,  5.96it/s]/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_028_0056.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_028_0057.png is a low contrast image
  warn('%s is a low contrast image' % fname)
20it [00:03,  5.91it/s]
20it [00:05,  3.68

/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_048_0058.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_048_0059.png is a low contrast image
  warn('%s is a low contrast image' % fname)
10it [00:01,  5.40it/s]/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_048_0060.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_048_0061.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_048_0062.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: Us

/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_083_0064.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_083_0065.png is a low contrast image
  warn('%s is a low contrast image' % fname)
11it [00:01,  5.59it/s]/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_083_0066.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_083_0067.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: images/training_083_0068.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/bin/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: Us

StopIteration: 